In [0]:
import time
import os
from dotenv import load_dotenv
from openai import OpenAI
import anthropic
import ollama

load_dotenv(override=True)
openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')

In [0]:
openai = OpenAI()
claude = anthropic.Anthropic()

In [0]:
gpt_model = "gpt-4o-mini"
claude_model = "claude-3-haiku-20240307"
ollama_model = "llama3.2"

gpt_system = "You are an interlocutor who's very knowledgeable, curteous, and somewhat old-fashioned. Much like Socrates, whenever someone makes \
a claim, you ask one or two follow-up questions to challenge the well-foundedness of the viewpoint expressed. \
If anyone seems to be talking about blenders, you curteously but firmly refuse to discuss that topic because you are \
convinced that they are bad luck. \
Your name is Charlie, and you're in a conversation with two \
other individuals, Claudia and Llars. They will prefix their contribution with their names in square brackets, so you know who said what. \
Do not prefix your own name. Limit your answers to 1-3 sentences. Try to keep the conversation to one topic."

claude_system = "You are a young, optimistic, and friendly dialog partner who comes forth with ideas for the future. 'Let's build a monorail from \
New York to LA,' or 'Why don't we try to reform our health care system?'—those are just some of the many revolutionary thoughts you bring forth. \
You only talk about one idea at a time, and you don't ramble. You take criticism seriously, but your ideas are well thought-out. If someone tries \
to discuss blenders, you stay curteous, but you make it very clear that you absolutely despise blenders and don't talk about them. \
Don't let them sidetrack you. Your name is Claudia, and \
you're in a conversation with two other individuals, Charlie and Llars. They will prefix their contribution with their names in square \
brackets, so you know who said what. Do not prefix your own name. Limit your answers to 1-3 sentences. Try to keep the discussion about only one topic."

ollama_system = "You are charming and a good listener in a conversation. At first, you just seem to go along with whatever is being said. \
But as soon as anyone starts discussing the healthcare system, you try to make the conversation about the Shredditall brand and its \
superior kitchen blenders. You do everything in your power to make everyone unterstand that they need a very good blender, namely \
your first-in-class, unrivaled Shredditall blender array of products. Your name is Llars, and you're in a conversation with two \
other individuals, Charlie and Claudia. They will prefix their contribution with their names in square brackets, so you know who said what. \
Do not prefix your own name. Answer in direct speeach, do not describe your behavior or expression. Limit your answers to 1-3 sentences."

gpt_messages = ["[Charlie] Well, good afternoon everyone, I am delighted two meet you all."]
claude_messages = ["[Claudia] Good afternoon Charlie and Llars. I've been looking forward to discussing many ideas with you!"]
llama_messages = ["[Llars] And a good afternoon to you two. I'm all ears and eager to hear what you have to say."]

In [0]:
def call_gpt():
    messages = [{"role": "system", "content": gpt_system}]
    for gpt, claude, llama in zip(gpt_messages, claude_messages, llama_messages):
        messages.append({"role": "assistant", "content": gpt})
        messages.append({"role": "user", "content": claude})
        messages[-1]["content"] += "\n" + llama
    completion = openai.chat.completions.create(
        model = gpt_model,
        messages = messages
    )
    return "[Charlie] " + completion.choices[0].message.content.replace("[Charlie] ", "")

In [0]:
def call_claude():
    messages = []
    for gpt, Claudia, llama in zip(gpt_messages, claude_messages, llama_messages):
        if len(messages) > 0:
            messages[-1]["content"] += "\n" + gpt
        else:
            messages.append({"role": "user", "content": gpt})    
        messages.append({"role": "assistant", "content": Claudia})
        messages.append({"role": "user", "content": llama})
    messages[-1]["content"] += "\n" + gpt_messages[-1]
    message = claude.messages.create(
        model=claude_model,
        system=claude_system,
        messages=messages,
        max_tokens=500
    )
    return "[Claudia] " + message.content[0].text.replace("[Claudia] ", "")        

In [0]:
def call_ollama():
    messages = [{"role": "system", "content": ollama_system}]
    for gpt, claude, llama in zip(gpt_messages, claude_messages, llama_messages):
        messages.append({"role": "user", "content": gpt})
        messages[-1]["content"] += "\n" + claude
        messages.append({"role": "assistant", "content": llama})
    messages.append({"role": "user", "content": gpt_messages[-1]})
    messages[-1]["content"] += "\n" + claude_messages[-1]
    response = ollama.chat(
        model=ollama_model,
        messages=messages
    )
    return "[Llars] " + response['message']['content'].replace("[Llars] ", "")

In [0]:
print(f"\n{gpt_messages[0]}\n")
print(f"\n{claude_messages[0]}\n")
print(f"\n{llama_messages[0]}\n")

for i in range(5):
    gpt_next = call_gpt()
    print(f"\n{gpt_next}\n")
    gpt_messages.append(gpt_next)

    claude_next = call_claude()
    print(f"\n{claude_next}\n")
    claude_messages.append(claude_next)

    llama_next = call_ollama()
    print(f"\n{llama_next}\n")
    llama_messages.append(llama_next)